KEYWORD EXTRACTION

In [1]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt') 

from rake_nltk import Rake
rake_nltk_var = Rake()
text = """frog and dog are playing"""
rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

['playing', 'frog', 'dog']


PARTS OF SPEECH TAGGING

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

tokenized = word_tokenize(text)

tagged = nltk.pos_tag(tokenized)
print(tagged)

[('frog', 'NN'), ('and', 'CC'), ('dog', 'NN'), ('are', 'VBP'), ('playing', 'VBG')]


SENTIMENT ANALYSIS

In [3]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
def retSetiment(text1):
    analyzer = SentimentIntensityAnalyzer()

    user_input = text1
    
    sentiment_scores = analyzer.polarity_scores(user_input)

    if sentiment_scores['compound'] >= 0.05:
        sentiment = "positive"
    elif sentiment_scores['compound'] <= -0.05:
        sentiment = "negative"
    else:
        sentiment = "neutral"

    print(f"Sentiment: {sentiment}")
    return sentiment

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


ADDING SUBJECT DESCRIPTORS ( ENHANCERS )

In [4]:
import json
import re
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

def appendAdj(extNoun):
    extNoun=extNoun.lower()
    animals_adj = open("./data.json")
    ani_json = json.load(animals_adj)
    name_adj_dict = {}
    ani_json = ani_json['animals']
    sentSentiment = retSetiment(text) #negative
    currAdj = []
    for i in ani_json:
        if(i['name'] == extNoun):
            for j in  range(len(i['adjectives'])):
                if(i['adjectives'][j]["type"] == sentSentiment):
                    currAdj.append(i['adjectives'][j]["value"])
                    
    name_adj_dict[extNoun] = currAdj
   
    return name_adj_dict[extNoun]
    
    
    
allAdj = {}
allText = []

k=0

for i,j in (tagged):
    if(j=='NN'):
        allAdj[i] = appendAdj(i)
# for i in allAdj:
#     print(i," ",allAdj[i])
allText.append([])
allText[0]=[text]
length=1
for k in allAdj:
    allText.append([])
    if(allAdj[k] == []):
        continue
    for v1 in allAdj[k]:
        for m in allText[length-1]:
            v1 += " " + k
            allText[length].append(re.sub(k,v1,m))
    length+=1
for i in allText[length-1]:
    print(i+"\n")

e:\FYP\prompt_generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sentiment: positive
Sentiment: positive
colorful frog and cute dog are playing

vibrant frog and cute dog dog are playing

striking frog and cute dog dog dog are playing

tropical frog and cute dog dog dog dog are playing

colorful frog and active dog are playing

vibrant frog and active dog dog are playing

striking frog and active dog dog dog are playing

tropical frog and active dog dog dog dog are playing

colorful frog and happy dog are playing

vibrant frog and happy dog dog are playing

striking frog and happy dog dog dog are playing

tropical frog and happy dog dog dog dog are playing

colorful frog and obedient dog are playing

vibrant frog and obedient dog dog are playing

striking frog and obedient dog dog dog are playing

tropical frog and obedient dog dog dog dog are playing



HABITAT MAPPING AND EMBEDDING

In [6]:
import json
import joblib

fdata = open('./data.json')
fdata = json.load(fdata)
fdata = fdata['animals']

def checkAndReturnHabitat(extNoun):
    for i in fdata:
        if(i['name'] == extNoun):
            # print(i['habitats'])
            return i['habitats']
    return []

def isHabitat(extNoun):
    model = joblib.load('habitatModel.pkl')
    vectorizer = joblib.load('vectorModel.pkl')

    user_input = extNoun

    user_input_preprocessed = user_input

    user_input_vec = vectorizer.transform([user_input_preprocessed])

    prediction = model.predict(user_input_vec)
    if prediction[0] == 1:
        return True
    return False


#habitat inclusion
currAllText = allText[length-1]
newAllText = []
setHabitats = []

flag = True
for i,j in (tagged):
    if (j == 'NN'):
        if(isHabitat(i) == False):
            flag = False
        else:
            flag = True
            break
print(flag)
#if there is no habitat found in the sentence
if flag == True:
    for i,j in (tagged):
        if ((j == 'NN') and (isHabitat(i) == False)): #dog
            setHabitats = checkAndReturnHabitat(i)
            if(setHabitats == []):
                continue
            for l in currAllText:
                for k in setHabitats:
                    temp = l
                    temp += ", in the " + k
                    newAllText.append(temp)
    for i in newAllText:
        print(i,end = "\n")

True
colorful frog and cute dog are playing, in the rainforests
colorful frog and cute dog are playing, in the jungles
colorful frog and cute dog are playing, in the tropical forests
vibrant frog and cute dog dog are playing, in the rainforests
vibrant frog and cute dog dog are playing, in the jungles
vibrant frog and cute dog dog are playing, in the tropical forests
striking frog and cute dog dog dog are playing, in the rainforests
striking frog and cute dog dog dog are playing, in the jungles
striking frog and cute dog dog dog are playing, in the tropical forests
tropical frog and cute dog dog dog dog are playing, in the rainforests
tropical frog and cute dog dog dog dog are playing, in the jungles
tropical frog and cute dog dog dog dog are playing, in the tropical forests
colorful frog and active dog are playing, in the rainforests
colorful frog and active dog are playing, in the jungles
colorful frog and active dog are playing, in the tropical forests
vibrant frog and active dog do

FINDING COSINE SIMILARITY

In [7]:
def cosineTop10(allText1):
    maxDict1 = {}
    top10Sentences = []
    for i in allText1:
        col1=[i]
        col2=[text]
        vectors1 = model.encode(col1, convert_to_tensor=True)
        vectors2 = model.encode(col2, convert_to_tensor=True)
        cosine_scores = util.cos_sim(vectors1, vectors2)
        for j,(sent1,sent2) in enumerate(zip(col1,col2)):
            if cosine_scores[j][j]>=0.5:
                maxDict1[i] = cosine_scores[j][j]
    l1 = 10
    ind1 = 0
    for k, v in sorted(maxDict1.items(), key=lambda item: item[1],reverse=True):
        if(ind1 >= l1 or ind1 >= len(maxDict1)):
            break
        ind1+=1
        top10Sentences.append(k)
    return top10Sentences 
topSentences = cosineTop10(newAllText)
for i in topSentences:
    print(i,end = "\n")
    

colorful frog and active dog are playing, in the jungles
striking frog and active dog dog dog are playing, in the jungles
striking frog and cute dog dog dog are playing, in the jungles
colorful frog and active dog are playing, in the rainforests
colorful frog and cute dog are playing, in the jungles
striking frog and happy dog dog dog are playing, in the jungles
colorful frog and happy dog are playing, in the jungles
vibrant frog and active dog dog are playing, in the jungles
colorful frog and happy dog are playing, in the rainforests
tropical frog and active dog dog dog dog are playing, in the jungles


KNOWLEDGE GRAPH GENERATION - BASIC GoT

In [8]:
class Node:
    def __init__(self, name, node_type):
        self.name = name
        self.node_type = node_type
        self.children = []
    def add_child(self, child):
        self.children.append(child)
    def __str__(self) -> str:
        return f"{self.name} - ({self.node_type})"
   
class Graph:
    def __init__(self):
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, parent, child):
        parent.add_child(child)

    def get_children(self, node):
        return node.children
    def findNode(self,animalName):
        for i in self.nodes:
            if(i.node_type == 'animal' and i.name == animalName):
                temp =  self.get_children(i)
                return temp
        return []
# Create a graph
graph = Graph()
data = open('./data.json')

data_json = json.load(data)

data_json = data_json['animals']

root = Node("animal", "root")
graph.add_node(root)
for i in range(len(data_json)):
    n1 = Node(data_json[i]['name'],"animal")
    graph.add_node(n1)
    for adj in data_json[i]["adjectives"]:
        adjective_node = Node(adj, "adjective")
        graph.add_node(adjective_node)
        graph.add_edge(n1, adjective_node)

    for habitat in data_json[i]["habitats"]:
        habitat_node = Node(habitat, "habitat")
        graph.add_node(habitat_node)
        graph.add_edge(n1, habitat_node)

testAnimal = "dog"
cdren = graph.findNode(testAnimal)
if(cdren != []):
    for i in cdren:
        print(i)
else:
    print("Not Found")

{'value': 'cute', 'type': 'positive'} - (adjective)
{'value': 'angry', 'type': 'negative'} - (adjective)
{'value': 'ferocious', 'type': 'negative'} - (adjective)
{'value': 'active', 'type': 'positive'} - (adjective)
{'value': 'happy', 'type': 'positive'} - (adjective)
{'value': 'obedient', 'type': 'positive'} - (adjective)
rainforests - (habitat)
jungles - (habitat)
tropical forests - (habitat)


Follow Up:
    - image type - photo, sketch, cartoon...
    - camera features - focus, lens...
    - time of the scenario - morning, afternoon, night, dinner...
    - adding +,- weights